In [ ]:
"""
/*
This has been generated by the overpass-turbo wizard.
The original search was:
*/
[out:json][timeout:250];
// fetch area “Bielsko-Biała” to search in
(
  {{geocodeArea:Madrid}};
  {{geocodeArea:Parla}};
  {{geocodeArea:Mostoles}};
)->.searchArea;
// gather results
(
  // query part for: “pharmacy”
  node["place"="square"](area.searchArea);
  way["place"="square"](area.searchArea);
  relation["place"="square"](area.searchArea);
);
// print results
out body;
>;
out skel qt;
"""

In [ ]:
#FALTA CONVERTIR LAS COLUMNAS PROPIAS DE CADA DATASET A TAGS
#RECONOCER LAS FILAS POR TIPO DE GEOMETRIA Y EXPORTAR

In [ ]:
#import requests
#url = "https://download.geofabrik.de/europe/spain/madrid-latest.osm.pbf"

In [ ]:
#response = requests.get(url, stream=True)

In [ ]:
#open(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\temp\madrid-latest.osm.pbf", "wb").write(response.content)

In [ ]:
routes = ["bus", "ferry", "railway", "subway", "train", "tram", "trolleybus"]
rails = ["tramway", "light_rail", "rail", "subway", "tram"]
bus = ['yes']
queries = {
    'transit':{'route': routes, 'railway': rails, 'bus': bus, 'public_transport': True},
    'park' : {'leisure': ['park'],'landuse':['forest']}, 
    'square':{'place':['square'],'highway':['pedestrian']}
    }

In [ ]:
from pyrosm import OSM
from pyrosm import get_data
fp = get_data("madrid")

# Initialize the OSM parser object with test data from Helsinki
osm = OSM(fp)

osm_data = {key:osm.get_data_by_custom_criteria(custom_filter=query,
                                        filter_type="keep",
                                        keep_nodes=True, 
                                        keep_ways=True, 
                                        keep_relations=True) for key,query in queries.items()}

In [ ]:
cols = ['id','timestamp','lon','lat','category','name','tags','osm_type','geometry']
for key, data in osm_data.items():
    print(key)
    data['category'] = key
    if 'name' not in data.columns:
        import ast
        data['name'] = data['tags'].fillna('{"no":"no"}').apply(ast.literal_eval).str['name']
        
    osm_data[key] = data[cols]

In [ ]:
import pandas as pd
c = pd.concat(osm_data.values())

In [ ]:
geom = c[c['id']==4263036]['geometry'].iloc[0]

In [ ]:
from shapely.geometry import LineString, MultiLineString

In [ ]:
def isring(geom):
    if isinstance(geom,MultiLineString):
        if tuple(round(x,5) for x in geom.geoms[0].coords[0]) == tuple(round(x,5) for x in geom.geoms[-1].coords[-1]):
            return True
        else:
            return False
    elif isinstance(geom,LineString):
        if tuple(round(x,5) for x in geom.coords[0]) == tuple(round(x,5) for x in geom.coords[-1]):
            return True
        else:
            return False
    else:
        False
    

In [ ]:
closering(geom)

In [ ]:
c = c.loc[(c['category'].isin(['square','park']))&(c['geometry'].apply(lambda g: isring(g)))].assign(
    geometry = lambda row: row['geometry'].convex_hull)
                                                                       
                                                                       
#.isin(['LineString','MultiLineString']))].assign(
 #   geometry = lambda row: row['geometry'].convex_hull)


In [ ]:
#c[(c['name'].fillna("").str.contains('Plaza Mayor'))&(c['category']=='square')]
c[c['id']==4263036]['geometry']

In [ ]:
c.to_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\temp\test_osm.gpkg", driver = 'GPKG')

In [ ]:

c.to_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\temp\test_osm.gpkg", driver = 'GPKG')
#c[c['osm_type']=='node'].to_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\temp\points_osm.gpkg", driver = 'GPKG')
#c[c['osm_type']=='relation'].to_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\temp\polygons_osm.gpkg", driver = 'GPKG')
#c[c['osm_type']=='way'].to_file(r"C:\Users\ManuBenito\Documents\Walknet-DataLake\temp\ways_osm.gpkg", driver = 'GPKG')

In [ ]:
osm_data['transit'].dtypes

In [ ]:
osm_data['park'].columns

In [ ]:
import ast
osm_data['square']['tags'].fillna('{"no":"no"}').apply(ast.literal_eval).str['name']

In [ ]:
osm_data['park'].fillna("").query('name.str.contains("Retiro")', engine='python')
#df.query('column_name.str.contains("abc")', engine='python')


In [ ]:
#file = r"C:\Users\ManuBenito\Documents\Walknet-DataLake\temp\madrid-latest.osm.pbf"

In [ ]:
# Read natural
# ============
from pyrosm import OSM
from pyrosm import get_data
fp = get_data("madrid")
# Initialize the OSM parser object
osm = OSM(fp)
#natural = osm.get_natural()
#natural.plot(column='natural', legend=True, figsize=(10,6))

In [ ]:
from pyrosm import OSM
from pyrosm import get_data
fp = get_data("madrid")

# Initialize the OSM parser object with test data from Helsinki
osm = OSM(fp)

# Test reading all transit related data (bus, trains, trams, metro etc.)
# Exclude nodes (not keeping stops, etc.)
routes = ["bus", "ferry", "railway", "subway", "train", "tram", "trolleybus"]
rails = ["tramway", "light_rail", "rail", "subway", "tram"]
bus = ['yes']
transit = osm.get_data_by_custom_criteria(custom_filter={
                                        'route': routes,
                                        'railway': rails,
                                        'bus': bus,
                                        'public_transport': True},
                                        # Keep data matching the criteria above
                                        filter_type="keep",
                                        # Do not keep nodes (point data)    
                                        keep_nodes=False, 
                                        keep_ways=True, 
                                        keep_relations=True)

In [ ]:
from pyrosm import OSM
from pyrosm import get_data
fp = get_data("madrid")

# Initialize the OSM parser object with test data from Helsinki
osm = OSM(fp)

In [ ]:
parks = osm.get_data_by_custom_criteria(custom_filter={
                                        'leisure': ['park'],
                                        'place':['square']},
                                        filter_type="keep",
                                        # Do not keep nodes (point data)    
                                        keep_nodes=True, 
                                        keep_ways=True, 
                                        keep_relations=True)

In [ ]:
parks.plot(column='name')

In [ ]:
parks.columns

In [ ]:
for p in parks['name'].unique():
    print(p)